💻 **준비 코드**

In [1]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('gender_submission.csv')

for df in [train, test]:
    df['Gender'] = df['Sex'].map({'male': 0, 'female': 1})
    df.loc[df['Embarked'].isnull(), 'Embarked'] = 'S'

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

# 3. 사라진 요금(Fare)을 찾아서: 숫자형 결측치는 어떻게 채워야 할까?

이제 승객들이 지불한 요금(Fare) 데이터를 살펴보겠습니다. 요금은 앞서 사용했던 수치형 변수 중 하나지만, test 데이터에 결측치가 존재하여 추가적인 처리가 필요합니다. 결측치를 어떤 값으로 채워야 할까요?

생각해보면 요금은 객실 등급(Pclass)과 밀접한 관련이 있을 것 같습니다. 게다가 같은 등급의 객실이라도 탑승 항구(Embarked)에 따라 요금이 달랐을 수 있죠. 이러한 관계들을 분석하여 가장 적절한 값으로 결측치를 채워보도록 하겠습니다.

준비가 되셨나요? 그럼 Fare 변수를 자세히 들여다보면서, 결측치를 채우기 위한 여정을 시작해보겠습니다.



## Fare 결측치 현황 파악

먼저 train 데이터와 test 데이터에서 Fare 변수의 결측치가 얼마나 있는지 확인해보겠습니다. 결측치의 수를 파악하는 것은 데이터 전처리의 첫 단계입니다.

📝 **프롬프트**
```
train과 test 데이터의 Fare 컬럼 결측치 개수를 계산해줘
```

💻 **코드 & 실행결과**

In [ ]:
print(f"Number of missing values in Fare (train): {train['Fare'].isnull().sum()}")
print(f"Number of missing values in Fare (test): {test['Fare'].isnull().sum()}")

이 코드는 Fare(요금) 컬럼의 결측치 개수를 확인합니다:
- `isnull()`은 각 값이 결측치인지 여부를 확인합니다
- `sum()`은 결측치의 총 개수를 계산합니다
- f-string을 사용하여 결과를 보기 좋게 출력합니다

실행 결과를 보면, train 데이터에는 Fare 결측치가 없고 test 데이터에는 1개의 결측치가 있음을 알 수 있습니다. test 데이터의 결측치는 비록 하나지만, 정확한 예측을 위해서는 적절한 값으로 채워넣는게 좋습니다.



## 결측치 승객 정보 확인

요금이 누락된 승객은 어떤 사람일까요? 이 승객의 다른 정보들을 살펴보면 적절한 요금을 추정하는데 도움이 될 수 있습니다.

📝 **프롬프트**
```
Fare가 결측치인 승객의 모든 정보를 보여줘
```

💻 **코드 & 실행결과**

In [ ]:
test[test.Fare.isna()]

이 코드는 test 데이터에서 Fare가 결측치인 승객 (Storey, Mr. Thomas)의 정보를 조회합니다:
- `test.Fare.isna()`로 Fare가 결측치인 행을 True로 표시합니다
- 이 조건을 test 데이터프레임의 인덱싱에 사용하여 해당하는 행을 선택합니다

실행 결과를 보면 결측치를 가진 승객의 특징을 알 수 있습니다:
- 3등석(Pclass=3) 승객입니다
- 남성(Sex='male') 승객입니다
- Southampton(Embarked='S')에서 탑승했습니다
- 혼자 탑승했습니다(SibSp=0, Parch=0)

이러한 정보는 결측치를 채울 때 매우 유용할 것 같습니다. 특히 같은 등급, 같은 항구에서 탑승한 다른 승객들의 요금을 참고하면 좋을 것 같네요.



## 등급과 항구별 승객 통계 분석

결측치를 채우기 위해 좀 더 자세한 통계를 살펴보겠습니다. 객실 등급(Pclass)과 탑승 항구(Embarked)의 각 조합별로 승객 수와 중간 요금을 함께 확인하면, 더 신뢰할 수 있는 참고 값을 얻을 수 있을 것입니다.

📝 **프롬프트**
```
객실 등급과 탑승 항구별로 그룹을 나누고, 각 그룹의 승객 수와 요금 중앙값을 계산해줘
```

💻 **코드 & 실행결과**

In [ ]:
passenger_stats = train.groupby(['Pclass', 'Embarked'])['Fare'].agg(['count', 'median'])
passenger_stats

이 코드는 다음과 같은 상세 분석을 수행합니다:
- `groupby(['Pclass', 'Embarked'])`로 객실 등급과 탑승 항구 조합별로 그룹을 만듭니다
- `agg(['count', 'median'])`으로 각 그룹에 대해 두 가지 계산을 수행합니다:
  - 'count': 각 그룹의 승객 수를 세어줍니다
  - 'median': 각 그룹의 요금 중앙값을 계산합니다

실행 결과를 보면 각 조합에 대한 상세 정보를 알 수 있습니다:
- 승객 수를 통해 각 그룹의 표본 크기를 확인할 수 있습니다
- 특히 3등석 Southampton의 경우 가장 많은 승객 수를 보여 중앙값의 신뢰도가 높을 것으로 예상됩니다
- 중앙값을 보면 같은 등급이라도 탑승 항구에 따라 요금 차이가 있음을 알 수 있습니다

이 통계를 바탕으로 결측치를 채울 적절한 값을 결정할 수 있을 것 같습니다.



## 결측치 채우기

test 데이터에서 발견된 결측치를 채워보겠습니다. 그런데 잠깐, 여기서 한 가지 의문이 들 수 있습니다.

"test 데이터의 결측치를 채우는데, 왜 train 데이터의 중앙값을 계산해서 채우지? test 데이터의 중앙값 또는 전체 데이터의 중앙값을 사용해야 되는 것 아냐?"

이것은 머신러닝에서 매우 중요한 원칙과 관련이 있습니다. 바로 '데이터 유출'을 방지하기 위한 것이죠. 자세히 알아볼까요?



:::{admonition} 왜 train 데이터로 결측치를 채워야 할까?
:class: tip

머신러닝 프로젝트에서는 test 데이터의 결측치를 채울 때 반드시 train 데이터를 기준으로 해야 합니다. 그 이유는 크게 두 가지입니다:

1. **데이터 유출(Data Leakage) 방지**
   - 데이터 유출이란 모델이 '미래의 정보'를 미리 알게 되는 상황을 말합니다
   - 예를 들어 봅시다. 여러분이 시험을 준비한다고 가정해볼까요?
     - 올바른 방법: 기출문제(train data)로 공부하고 실제 시험(test data)을 봅니다
     - 잘못된 방법: 실제 시험지를 미리 보고 그걸 참고해서 공부합니다
   - test 데이터의 정보를 사용하여 결측치를 채우는 것은 두 번째 경우와 같습니다
   - 이는 마치 아직 일어나지 않은 미래의 정보를 사용하는 것과 같은 '부정행위'입니다

2. **실전 상황 고려**
   - 실제 서비스 환경을 생각해봅시다
   - 예를 들어 내일 타이타닉호에 새로운 승객이 탑승한다면?
   - 그 승객의 요금 정보가 누락되었다면?
   - 당연히 과거의 데이터(train data)를 참고해서 채워야겠죠
   - test 데이터는 이런 '미래의 새로운 승객'을 시뮬레이션하는 것입니다

이러한 원칙은 결측치 처리뿐만 아니라 모든 종류의 데이터 전처리 과정에서 지켜져야 합니다. test 데이터는 오직 최종 예측을 위해서만 사용되어야 하며, 어떠한 통계값 계산에도 활용되어서는 안 됩니다.
:::



이제 Fare의 결측치를 채우겠습니다.

📝 **프롬프트**
```
1. 3등석이면서 Southampton에서 탑승한 승객들의 요금 중앙값을 구해서
2. 그 값으로 결측치를 채운 후
3. 해당 승객(PassengerId=1044)의 정보를 다시 출력해줘
```

💻 **코드 & 실행결과**

In [ ]:
median_fare = train[(train['Pclass'] == 3) & (train['Embarked'] == 'S')]['Fare'].median()
test['Fare'] = test['Fare'].fillna(median_fare)

test[test['PassengerId'] == 1044]

이 코드는 세 가지 작업을 순차적으로 수행합니다:
1. 먼저 train 데이터에서 동일 조건(3등석, Southampton)의 승객들의 중간 요금을 계산합니다
2. 계산된 중간 요금으로 test 데이터의 결측치를 채웁니다
3. 결과를 확인합니다



## 다른 가능성

여기서 한 가지 더 생각해볼 것이 있겠네요. 우리는 지금까지 Storey의 누락된 요금을 다른 승객들의 중앙값으로 채웠습니다. 하지만 혹시 이 결측치에는 다른 의미가 있는 것은 아닐까요?

예를 들어, 요금을 지불하지 않은 승객의 경우 0이라고 기록하는 대신 공란으로 남겨두었을 수도 있지 않을까요? 실제로 그런 사례가 있었는지, 있었다면 Storey의 케이스와 어떤 연관이 있을지 궁금해집니다.

이러한 의문을 가지고 다음 섹션에서는 조금 더 흥미로운 가설을 세워보겠습니다. 무임승객이 실제로 존재했는지, 만약 있었다면 어떤 특징이 있었는지 살펴보면서, Storey의 결측치에 대한 새로운 해석을 시도해보도록 하겠습니다.